# 1  🪬 Aircraft and engines
he prop package in openap allows users to obtain aircraft and engine-related data quickly. We will demonstrate usage through the following example.
## 1.1 Supported aircraft type codes
First, we need to import the prop package from the OpenAP library.

In [5]:
from openap import prop
from pprint import pprint

We can get a list of available aircraft openap.prop.available_aircraft() function:

In [6]:
avaiable_aircraft = prop.available_aircraft()

print(f"Supports {len(avaiable_aircraft)} aircraft types")
print(avaiable_aircraft)

Supports 37 aircraft types
['a19n', 'a20n', 'a21n', 'a318', 'a319', 'a320', 'a321', 'a332', 'a333', 'a343', 'a359', 'a388', 'b37m', 'b38m', 'b39m', 'b3xm', 'b734', 'b737', 'b738', 'b739', 'b744', 'b748', 'b752', 'b763', 'b772', 'b773', 'b77w', 'b788', 'b789', 'c550', 'crj9', 'e145', 'e170', 'e190', 'e195', 'e75l', 'glf6']


## 1.2 Aircraft data
We can get the parameters for one aircraft using openap.prop.aircraft() function:

In [7]:
aircraft = prop.aircraft("A320")
pprint(aircraft)

{'aircraft': 'Airbus A320',
 'ceiling': 12500,
 'cruise': {'height': 11000, 'mach': 0.78, 'range': 5000},
 'drag': {'cd0': 0.018, 'e': 0.799, 'gears': 0.017, 'k': 0.039},
 'engine': {'default': 'CFM56-5B4',
            'mount': 'wing',
            'number': 2,
            'options': {'A320-111': 'CFM56-5-A1',
                        'A320-211': 'CFM56-5-A1',
                        'A320-212': 'CFM56-5A3',
                        'A320-214': 'CFM56-5B4',
                        'A320-215': 'CFM56-5B5',
                        'A320-216': 'CFM56-5B6',
                        'A320-231': 'V2500-A1',
                        'A320-232': 'V2527-A5',
                        'A320-233': 'V2527E-A5'},
            'type': 'turbofan'},
 'flaps': {'Sf/S': 0.17,
           'area': 21.1,
           'bf/b': 0.78,
           'cf/c': 0.176,
           'lambda_f': 0.9,
           'type': 'single-slotted'},
 'fuel': {'engine': 'CFM56-5B4/P', 'fuel_coef': 2.65942225},
 'fuselage': {'height': 4.14, 'lengt

## 1.3 Engine data
The engine data can be obtained with openap.prop.engine() function:

In [4]:
engine = prop.engine("CFM56-5B4")
engine

{'uid': '2CM014',
 'name': 'CFM56-5B4',
 'manufacturer': 'CFM International',
 'type': 'TF',
 'bpr': 5.9,
 'pr': 27.1,
 'max_thrust': 117900.0,
 'ei_hc_to': 0.1,
 'ei_hc_co': 0.1,
 'ei_hc_app': 0.13,
 'ei_hc_idl': 3.87,
 'ei_co_to': 0.5,
 'ei_co_co': 0.5,
 'ei_co_app': 2.33,
 'ei_co_idl': 31.9,
 'ei_nox_to': 28.7,
 'ei_nox_co': 23.3,
 'ei_nox_app': 10.0,
 'ei_nox_idl': 4.3,
 'ff_to': 1.166,
 'ff_co': 0.961,
 'ff_app': 0.326,
 'ff_idl': 0.107,
 'fuel_lto': 421.0,
 'cruise_thrust': 22241.0,
 'cruise_sfc': 0.0154,
 'cruise_mach': 0.8,
 'cruise_alt': 35000.0,
 'fuel_ch': np.float64(5.2e-07)}

# 2  ☯️ Drag and thrust
The OpenAP provides two modules, drag and thrust, for the calculation of drag force and maximum available thrust force during the flight.
## 2.1 Compute the aircraft drag
he drag calculation is based on the OpenAP’s drag polar model, which is obtained based on open trajectory data and methodology from the paper: Sun et al. (2020).

Knowing the drag polar coefficients - 
 (zero-lift drag coefficient) and 
 (lift-induced drag coefficient) - of the aircraft, the drag force can be calculated based on the point-mass aircraft performance model:

 
 
 
 

When an aircraft is climbing and descending, the flight path angle is also considered to calculate the lift force. Hence, by including the vertical speed, the estimation of drag can be different.

An example calculation using the OpenAP’s drag module is:




In [5]:
from openap.drag import Drag

typecode = "A320"

mass = 62_000  # kg
TAS = 250  # kts
ALT = 20_000  # ft
VS = 1000  # ft/min

drag = Drag(ac=typecode)

# clean configuration
D = drag.clean(mass=mass, tas=TAS, alt=ALT, vs=VS)

print(f"""{typecode} at condition: 
Mass:{mass}  TAS:{TAS}  ALT:{ALT}  VS:{VS} 
Clean configuration 

Total drag is {D//1000} kN
""")

A320 at condition: 
Mass:62000  TAS:250  ALT:20000  VS:1000 
Clean configuration 

Total drag is 33.0 kN



The drag.clean() function estimates the drag force when the aircraft is at the clean configuration, which means no flaps or landing gear are deployed.

During the initial climb and approach, we can also calculate drag considering flaps (providing the flaps setting angle in degrees) and whether the landing gears are extended.

In [6]:
mass = 62_000  # kg
TAS = 150  # kts
ALT = 1000  # ft
VS = 1500  # ft/min
flap_angle = 20  # degree

# with flaps and landing gears
D = drag.nonclean(
    mass=mass,
    tas=TAS,
    alt=ALT,
    flap_angle=flap_angle,
    vs=VS,
    landing_gear=True,
)

print(f"""{typecode} at condition: 
Mass:{mass}  TAS:{TAS}  ALT:{ALT}  VS:{VS} 
Flap:{flap_angle} deg,  Landing Gear extended 

Total drag is {D//1000} kN
""")

A320 at condition: 
Mass:62000  TAS:150  ALT:1000  VS:1500 
Flap:20 deg,  Landing Gear extended 

Total drag is 46.0 kN



## 2.2 Compute maximum aircraft engine thrust
OpenAP implements the engine thrust model proposed by Bartel & Young (2008). Small adjustments to the model was made to improve the computational efficiency.

To calculate the maximum net thrust of an aircraft, we first instantiate the thrust.Thrust object:

In [7]:
from openap.thrust import Thrust

thr_a320 = Thrust(ac="A320", eng="CFM56-5B4")

The function Thrust.takeoff() function is used to calculate maximum thrust during the take-off at different speed (kts) and altitude (ft) conditions, for example:

In [8]:
T = thr_a320.takeoff(tas=100, alt=0)
print(f"Max thrust: {round(T/1000, 2)} kN")

Max thrust: 200.72 kN


The Thrust.climb() function estimates the maximum net thrust during the climb, which requires TAS (kts), altitude (ft), and rate of climb (ft/min).

An example for A320 climbing at 1000 ft/min at 10,000 ft with TAS 250 kts:



In [9]:
T = thr_a320.climb(tas=250, alt=10000, roc=1000)
print(f"Max thrust: {round(T/1000, 2)} kN")

Max thrust: 89.23 kN


The Thrust.cruise() function estimates the maximum net thrust during the cruise at different speeds and altitudes. An example is shown as follows.

In [10]:
T = thr_a320.cruise(tas=300, alt=32000)
print(f"Max thrust: {round(T/1000, 2)} kN")

Max thrust: 53.68 kN


## 2.3 Idle thrust
Furthermore, we can use the Thrust.descent_ide() function to estimate the idle thrust during the descent at different altitude and speed conditions.

The idle thrust is modeled as approximately 7% of the maximum thrust at the same altitude and speed.

An example is shown as follows:

In [18]:
T = thr_a320.descent_idle(tas=270, alt=30000)
print(f"Idle thrust: {round(T/1000, 2)} kN")

Idle thrust: 4.53 kN
